In [1]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import pandas as pd
from collections import defaultdict
import re

In [2]:
def hashnode():
    url = 'https://hashnode.com/community?source=header'
    html_content = requests.get(url)
    soup = BeautifulSoup(html_content.text, 'lxml')
    container = soup.find('div', attrs={'data-state':'active'})
    return container

In [3]:
def hashnode_data_points(html_body):
    all_articles = []
    
    for cont in html_body.find('div'):
        articles = cont.find('div', {'class':'flex flex-col gap-1 '})
    
    #extract article titles
        if cont.find('div', {'class':'flex flex-col gap-1'}) != None:
            article_titles = cont.find('h1').text

        #extract article authors name
        if cont.find('div', {'class': 'flex flex-row items-center justify-start gap-3'}) != None:
            names = cont.find('span', {'class':'font-semibold text-slate-700 dark:text-slate-200 cursor-pointer'}).text


        if cont.find('div', {'class': 'flex flex-row items-center justify-start gap-3'}) != None:
            dates = cont.find('p', {'class':'text-sm text-slate-500 dark:text-slate-400 font-normal'}).text


        #extracting the tag holding the article links    
        if cont.find('div', {'class': 'flex flex-row items-center justify-start gap-1'}) != None:
            article_links = cont.find('div', {'class': 'flex flex-row items-center justify-start gap-1'}).find_all('a')
            if len(article_links) > 1:
                second_links = article_links[1]['href']


        #extracting the tag holding all likes and reads of the article
        if cont.find('section', {'class':'flex flex-col gap-5'}) != None:
            reactions = cont.find('section', {'class':'flex flex-col gap-5'}).find_all('p')
            if len(reactions) > 3:
                likes = reactions[1].text
                reads = reactions[3].text


        if cont.find('div', {'class':'hidden md:block'}) != None:
            article_contents = cont.find('span', {'class':'text-base hidden font-normal text-slate-500 dark:text-slate-400 hn-break-words cursor-pointer md:line-clamp-3'})            

    #append all extracted tags object 
        all_articles.append([article_titles, names, dates, second_links, likes, reads])
    return all_articles
    
        

In [4]:
hashnode_result = hashnode()

In [5]:
articles_dict_list=defaultdict(list)
articles_colnames = ['Title', 'AuthorName', 'DatePosted', 'ArticleLink', 'Likes', 'Reads']

In [6]:
articles_df = pd.DataFrame(hashnode_data_points(hashnode_result), columns=articles_colnames)

In [7]:
articles_df

,Title,AuthorName,DatePosted,ArticleLink,Likes,Reads
0,Setting Up Post Schedules with EventBridge Sch...,Sandro Volpicella,"Jan 16, 2024",https://engineering.hashnode.com/setting-up-po...,84 likes,722 reads
1,Rust 🦀 Hello world,Francesco Ciulla,"Jan 10, 2024",https://blog.francescociulla.com/rust-hello-world,110 likes,950 reads
2,Rust 🦀 Hello world,Francesco Ciulla,"Jan 10, 2024",https://blog.francescociulla.com/rust-hello-world,110 likes,950 reads
3,Testing a React App Built with Vite Using Jest...,Sean Coughlin,5 hours ago,https://blog.seancoughlin.me/testing-a-react-a...,110 likes,950 reads
4,How To Create A Custom 404 Page In Next Js App...,Precious Egwuenu,12 hours ago,https://preshdev.hashnode.dev/how-to-create-a-...,21 likes,29 reads
5,Why AWS (Amazon web Services ?,Utkarsh Pathak,9 hours ago,https://utkarshpathak.hashnode.dev/why-aws-ama...,21 likes,29 reads
6,Why AWS (Amazon web Services ?,Utkarsh Pathak,9 hours ago,https://utkarshpathak.hashnode.dev/why-aws-ama...,21 likes,29 reads
7,New HTML Control Lands in Safari,Alvaro Montoro,8 hours ago,https://alvaromontoro.hashnode.dev/new-html-co...,21 likes,29 reads
8,APEX_WEB_SERVICE The Definitive Guide,Jon Dixon,9 hours ago,https://blog.cloudnueva.com/apexwebservice-the...,10 likes,404 reads
9,APEX_WEB_SERVICE The Definitive Guide,Jon Dixon,9 hours ago,https://blog.cloudnueva.com/apexwebservice-the...,10 likes,404 reads


In [8]:
def article_content(links):
    request_link = requests.get(links)
    soup_content = BeautifulSoup(request_link.text, 'lxml')
    content = soup_content.find('div', {'id':'post-content-wrapper'}).text
    return content


In [9]:
articles_df['ArticleContent'] = articles_df.apply(lambda row: article_content(row['ArticleLink']), axis=1)

In [10]:
# articles_df['ArticleContent'] = articles_df['ArticleContent'].replace(r'^\n', '', regex=True)
# articles_df['ArticleContent'] = articles_df['ArticleContent'].replace(r'\n', '\n\n', regex=True)

In [11]:
articles_df

,Title,AuthorName,DatePosted,ArticleLink,Likes,Reads,ArticleContent
0,Setting Up Post Schedules with EventBridge Sch...,Sandro Volpicella,"Jan 16, 2024",https://engineering.hashnode.com/setting-up-po...,84 likes,722 reads,One essential feature of any blogging platform...
1,Rust 🦀 Hello world,Francesco Ciulla,"Jan 10, 2024",https://blog.francescociulla.com/rust-hello-world,110 likes,950 reads,Rust is a systems programming language that ru...
2,Rust 🦀 Hello world,Francesco Ciulla,"Jan 10, 2024",https://blog.francescociulla.com/rust-hello-world,110 likes,950 reads,Rust is a systems programming language that ru...
3,Testing a React App Built with Vite Using Jest...,Sean Coughlin,5 hours ago,https://blog.seancoughlin.me/testing-a-react-a...,110 likes,950 reads,"In the world of modern web development, React ..."
4,How To Create A Custom 404 Page In Next Js App...,Precious Egwuenu,12 hours ago,https://preshdev.hashnode.dev/how-to-create-a-...,21 likes,29 reads,Introduction\n\nRedirecting to a page that is ...
5,Why AWS (Amazon web Services ?,Utkarsh Pathak,9 hours ago,https://utkarshpathak.hashnode.dev/why-aws-ama...,21 likes,29 reads,Agenda :\n\n\n\nWhat is Cloud ?\n\n\n\nPublic ...
6,Why AWS (Amazon web Services ?,Utkarsh Pathak,9 hours ago,https://utkarshpathak.hashnode.dev/why-aws-ama...,21 likes,29 reads,Agenda :\n\n\n\nWhat is Cloud ?\n\n\n\nPublic ...
7,New HTML Control Lands in Safari,Alvaro Montoro,8 hours ago,https://alvaromontoro.hashnode.dev/new-html-co...,21 likes,29 reads,Native toggle switches landed in one browser. ...
8,APEX_WEB_SERVICE The Definitive Guide,Jon Dixon,9 hours ago,https://blog.cloudnueva.com/apexwebservice-the...,10 likes,404 reads,Introduction\n\nAPEX_WEB_SERVICE is by far my ...
9,APEX_WEB_SERVICE The Definitive Guide,Jon Dixon,9 hours ago,https://blog.cloudnueva.com/apexwebservice-the...,10 likes,404 reads,Introduction\n\nAPEX_WEB_SERVICE is by far my ...


In [16]:
def convert_to_date(value):
    if 'hour' or 'an' in value:
        hours_ago = int(value.split()[0])
        return datetime.now() - timedelta(hours=hours_ago)
    elif 'minute' in value:
        minutes_ago = int(value.split()[0])
        return datetime.now() - timedelta(minutes=minutes_ago)
    else:
        try:
            return pd.to_datetime(value).date()
        except ValueError:
            return datetime.now().date()

In [17]:
articles_df['DatePosted'] = articles_df['DatePosted'].apply(convert_to_date)

ValueError: invalid literal for int() with base 10: 'Jan'

In [ ]:
articles_df